In [1]:
import core
import pandas as pd
from datetime import datetime
import ast

In [2]:
current_date = "2025-07-23"
datetime_obj = pd.to_datetime(current_date)

In [3]:
core.MonteCarloSim(match_id=4643)

Simulating Santos vs Internacional


Simulations (1 worker): 100%|██████████| 1/1 [00:00<?, ?it/s]


{'off_xg_coef': -0.000464926, 'def_xg_coef': 1.95818e-05, 'fouls_committed_rate': 0.0, 'fouls_drawn_rate': 0.0031746031746031746, 'yellow_card_rate': 0.0, 'red_card_rate': 0.0, 'sub_in_count': 0, 'sub_out_count': 0, 'in_status_prob': {'Leading': 0.3333333333333333, 'Level': 0.3333333333333333, 'Trailing': 0.3333333333333333}, 'out_status_prob': {'Leading': 0.3333333333333333, 'Level': 0.3333333333333333, 'Trailing': 0.3333333333333333}, 'initial_fatigue': 0.5032163898648545, 'initial_rhythm': 1, 'bench': False, 'on_field': True, 'yellow_card': False, 'red_card': False, 'in': 0}
{'off_xg_coef': 0.00017116, 'def_xg_coef': 7.58166e-05, 'fouls_committed_rate': 0.004032258064516129, 'fouls_drawn_rate': 0.0024193548387096775, 'yellow_card_rate': 0.2, 'red_card_rate': 0.0, 'sub_in_count': 0, 'sub_out_count': 2, 'in_status_prob': {'Leading': 0.3333333333333333, 'Level': 0.3333333333333333, 'Trailing': 0.3333333333333333}, 'out_status_prob': {'Leading': 0.4, 'Level': 0.2, 'Trailing': 0.4}, 'ini